In [2]:
import os
import json

Код для переведення даних open.sanctions в зручніший формат
З огляду на умову задачі, нам потрібні наступні поля: 
* індентифікатор компанії(невпевнений який конкретно, що означає але зберіг і ogrnCode, і innCode),
* назва компанії чи ім'я персони(name і caption на випадок, якщо першого немає),
* крізь кого пов'язані санкціями(з огляду структури об'єктів побачив, що об'єкт в наборі є підсанкційним, в його полях в полі properties зустрічається подібні речі: наприклад власники чи директори компанії)
Додатково зберігаю:
* id(щоб якщо на практиці потрібні були якісь не взяті нами поля, то можна було б їх швидко і зручно найти)
* список санкцій(аналогічно до id, але цей пункт на відміну від усіх інших полів нам швидше пригодиться)
При цьому якщо sanctions = None, тоді цей об'єкт зв'язаний з підсанкційним через related_to і навпаки.

In [104]:
def parse(obj, related):
    id = obj['id']
    caption = obj['caption']
    name = obj['properties']['name'] if ('name' in obj['properties']) else None
    ogrnCode = (obj['properties']['ogrnCode']) if ('ogrnCode' in obj['properties'].keys()) else None
    innCode = (obj['properties']['innCode']) if ('innCode' in obj['properties'].keys()) else None
    sanctions = str(obj['properties']['sanctions']) if ('sanctions' in obj['properties']) else None

    return {'id' : id, 'caption' : caption, 'name' : name, 'ogrnCode' : ogrnCode, 'innCode' : innCode, 'sanctions' : sanctions, 'related_to' : related}

In [109]:
counter = 0

with open('data\\opensanctions_nonbinary.json', 'r', encoding = 'utf8') as f:
    with open('data\\opensanctions_processed.json', 'w') as g:
        for line in f:
            obj = json.loads(line)
            try:
                parsed_obj = parse(obj, None)
            except:
                print('PARSING OBJ IS FAILED')
                print(obj)
            g.write(json.dumps(parsed_obj) + '\n')

            try:
                if('OwnerShip' in obj['properties'].keys()):
                    for ownership in obj['properties']['OwnerShip']:
                        for owner in ownership['properties']['owner']:
                            g.write(json.dumps(parse(owner, parsed_obj['id'])) + '\n')
            except:
                print('PARSING OWNERSHIP IS FAILED')
                print(obj)

            try:
                if('directorshipOrganization' in obj['properties'].keys()):
                    for directorship in obj['properties']['directorshipOrganization']:
                        for director in directorship['properties']['director']:
                            g.write(json.dumps(parse(director, parsed_obj['id'])) + '\n')
            except:
                print('PARSING DIRECTORSHIP IS FAILED')
                print(obj)

            try:
                if('ownershipAsset' in obj['properties'].keys()):
                    for ownership in obj['properties']['ownershipAsset']:
                        for owner in ownership['properties']['owner']:
                           g.write(json.dumps(parse(owner, parsed_obj['id'])) + '\n')
            except:
                print('PARSING OWNERSHIPASSET IS FAILED')
                print(obj)
                        
            counter += 1
            if(counter == 10):
                break

In [110]:
with open('data\\opensanctions_nonbinary.json', 'r', encoding = 'utf8') as f:
    print(f.readline())

{"id":"NK-224TRezPqwzhQZ37exWxtX","caption":"SANAVBARI NIKITENKO","schema":"Person","properties":{"birthPlace":["KOFARNIKHON TOWN"],"country":["tj"],"nationality":["ru"],"birthDate":["1992-06-28"],"gender":["female"],"sourceUrl":["https://ws-public.interpol.int/notices/v1/red/2019-108810"],"lastName":["NIKITENKO"],"firstName":["SANAVBARI"],"name":["SANAVBARI NIKITENKO"],"topics":["crime"],"sanctions":[{"id":"interpol-red-2d1a22ea375cce282336a0a398edea0edd4a2f19","caption":"Red Notice","schema":"Sanction","properties":{"country":["ru"],"authority":["INTERPOL"],"authorityId":["2019/108810"],"program":["Red Notice"],"entity":["NK-224TRezPqwzhQZ37exWxtX"],"sourceUrl":["https://www.interpol.int/How-we-work/Notices/View-Red-Notices"],"reason":["participation in the activity of a terrorist organization; participation in the activity of an illegal armed formation"]},"referents":[],"datasets":["interpol_red_notices"],"first_seen":"2023-06-06T16:26:42","last_seen":"2023-12-07T00:27:01","last_cha

Вдосконалений парсер з відстеженням помилок в коді і виводом їх

In [116]:
from json2txttree import json2txttree

In [131]:
with open('data\\opensanctions_nonbinary.json', 'r', encoding = 'utf8') as f:
    with open('data\\opensanctions_processed.json', 'w') as g:
        for line in f:
            obj = json.loads(line)
            try:
                parsed_obj = parse(obj, None)
                g.write(json.dumps(parsed_obj) + '\n')
            except Exception as error:
                print('PARSING OBJ IS FAILED')
                print(obj)
                print(json2txttree(json.loads(line.replace('[]', '[1]'))))
                print(error)
            

            try:
                if('OwnerShip' in obj['properties'].keys()):
                    for ownership in obj['properties']['OwnerShip']:
                        for owner in ownership['properties']['owner']:
                            g.write(json.dumps(parse(owner, parsed_obj['id'])) + '\n')
            except Exception as error:
                print('PARSING OWNERSHIP IS FAILED')
                print(obj)
                print(json2txttree(json.loads(line.replace('[]', '[1]'))))
                print(error)

            try:
                if('directorshipOrganization' in obj['properties'].keys()):
                    for directorship in obj['properties']['directorshipOrganization']:
                        for director in directorship['properties']['director']:
                            g.write(json.dumps(parse(director, parsed_obj['id'])) + '\n')
            except Exception as error:
                print('PARSING DIRECTORSHIP IS FAILED')
                print(obj)
                print(json2txttree(json.loads(line.replace('[]', '[1]'))))
                print(error)